In [11]:
import pdfplumber
import pandas as pd
import re
import numpy as np
from collections import defaultdict
from unidecode import unidecode

In [12]:
import pdfplumber
import pandas as pd
import re
import numpy as np
# from collections import defaultdict
from unidecode import unidecode

# --- Paso 1: Leer el PDF y extraer texto ---
def extract_text_from_pdf(pdf_path):
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_text = page.extract_text()
            if extracted_text:  # Asegurar que haya texto en la página
                text.append(extracted_text)
    return text

pdf_path = "/Users/sebastianmerino/Downloads/Los-miserables.pdf"
book_text = extract_text_from_pdf(pdf_path)

# Convertir a DataFrame y guardar como CSV
df = pd.DataFrame({"text": book_text})
#df.to_csv("los_miserables_raw.csv", index=False)

# --- Paso 2: Estandarizar y limpiar el texto ---
def standardize_text(text):
    text = text.lower()                    # Convertir a minúsculas
    text = unidecode(text)                  # Remover caracteres especiales
    text = re.sub(r'\W+', ' ', text)        # Remover puntuación/ no alfanumericos
    return text.strip()

# Aplicar la función para cada fila del DataFrame
df["text_cleaned"] = df["text"].apply(lambda x: standardize_text(x) if isinstance(x, str) else "")
# df.to_csv("los_miserables_cleaned.csv", index=False)

# --- Paso 3: Construir el vocabulario y calcular frecuencias ---
def build_vocabulary(text_series):
    vocabulary = {}  # Almacena las palabras y sus frecuencias

    for text in text_series.dropna():
        words = text.split()
        for word in words:
            if word in vocabulary:
                vocabulary[word] += 1  # Aumenta la frecuencia de palabras ya existentes
            else:
                vocabulary[word] = 1  # Añade nuevas palabras

    return vocabulary

# Crear el vocabulario a partir del texto limpio
vocab_dict = build_vocabulary(df["text_cleaned"])

# Convertir a DataFrame
vocab_df = pd.DataFrame(list(vocab_dict.items()), columns=["word", "frequency"])
vocab_df = vocab_df.sort_values(by="frequency", ascending=False)  # Ordenar por frecuencia

# Guardar en parquet
vocab_df.to_parquet("/Users/sebastianmerino/Downloads/Ejercicio_Los_miserables/vocabulary.parquet")

# --- Paso 4: Estadisticas ---
total_words = sum(vocab_dict.values())  # Total de palabras en el libro
unique_words = len(vocab_dict)  # Palabras únicas en el vocabulario
most_frequent_words = vocab_df.head(100)  # Top 100 palabras más frecuentes
words_above_threshold = vocab_df[vocab_df["frequency"] > 100]  # Palabras que aparecen +100 veces
words_with_freq_1 = vocab_df[vocab_df["frequency"] == 1]  # Palabras que aparecen solo una vez

# Resultados
print(f"Total de palabras en el libro: {total_words}")
print(f"Palabras únicas en el vocabulario: {unique_words}")
print("\nTop 10 palabras más frecuentes:")
print(most_frequent_words.head(10))

print(f"\nPalabras que aparecen más de 100 veces: {len(words_above_threshold)}")
print(f"Palabras que aparecen solo una vez: {len(words_with_freq_1)}")

Total words in book: 109947
Unique words in vocabulary: 13461

Top 10 Most Frequent Words:
   word  frequency
10   de       5328
24   la       3918
86  que       3818
56   el       3394
19    y       3123
20   en       2836
23    a       2490
6    se       1681
53   un       1601
79   no       1498

Words appearing more than 100 times: 105
Words appearing only once: 7493


In [7]:
vocab_df

,word,index
0,los,0
1,miserables,1
2,victor,2
3,hugo,3
4,https,4
...,...,...
15454,currucas,15454
15455,cortarla,15455
15456,angosta,15456
15457,ilegibles,15457
